%conda activate bert_envv

%conda deactivate

In [ ]:
import pandas as pd
import random
import os
import time
import openai
import re
import json


In [ ]:

#original dataset for one day of tweets
original_dataset = pd.read_csv('../../data/tweets/anon.tweets_2020-07-24_copy.csv')

#number of rows in the original dataset
num_rows_original = len(original_dataset)

num_random_rows = 1000

# 1000 random indices to make the model more accurate
random_indices = random.sample(range(num_rows_original), num_random_rows)

# new dataset 
new_dataset = original_dataset.iloc[random_indices]

# Save the new dataset to a new CSV file
new_dataset.to_excel('../../data/test/1000_random_rows_24_07_2020.xlsx', index=False)


In [ ]:



# Note: The openai-python library support for Azure OpenAI
# is in preview.
# Note: This code sample requires OpenAI Python library
# version 0.28.1 or lower. import os


# API settings for Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://computationalsocialsciences.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = 'ADD_KEY'

#os.getenv("OPENAI_API_KEY")
def get_true_or_false(input_tweet):
    prompt = f"Respond in the json format: {{'response': sentiment_classification}}\nText: {input_tweet}\nSentiment (true, false):"
    
    while True:
        try:
            completion = openai.ChatCompletion.create(
                engine="GPT3",
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                max_tokens=800,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None)
        except openai.error.InvalidRequestError as e:
            # Handle the specific error (rate limiting or content management policy)
            print(f"Error: {e}")

            # Check if the error includes information about how long to wait
            wait_match = re.search(r'(?<=Wait (\d+)s)', str(e))
            
            if wait_match:
                wait_time = int(wait_match.group(1))
                print(f"Waiting for {wait_time} seconds as per API request...")
                time.sleep(wait_time)
            else:
                # If no wait time is specified, sleep for a default time
                print("Waiting for 3 seconds...")
                time.sleep(3)
        else:
            # If no exception is raised, break the loop
            break
    
    response_text = completion.choices[0].message['content'].strip()
    match = re.search(r'\btrue\b|\bfalse\b', response_text, flags=re.IGNORECASE)
    
    if match:
        truthfulness = match.group(0).lower()
    else:
        truthfulness = "Not Found"

    return {"tweet": input_tweet, "truthfulness": truthfulness}


In [ ]:
sample_tweet = "Trump must have tossed you some paper towels."
result = get_true_or_false(sample_tweet)
print("Result\n", f"{result}")


Result
 {'tweet': 'Trump must have tossed you some paper towels.', 'truthfulness': 'false'}


In [ ]:
input_file_path = '1000_random_rows_24_07_2020.xlsx' #"practice_10_row_dataset.xlsx"


# Read the CSV file using pandas
df = pd.read_excel(input_file_path)

# Apply the function to the 'text' column and create a new column 'truthfulness'
df['truthfulness'] = df['text'].apply(lambda x: get_true_or_false(str(x).strip())['truthfulness'] if x else "")

display(df)

# Write the DataFrame to an XLSX file using openpyxl as the engine
df.to_excel("output_responses_1000.xlsx", index=False, engine='openpyxl')

RateLimitError: Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 39 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.

In [ ]:

# Note: The openai-python library support for Azure OpenAI
# is in preview.
# Note: This code sample requires OpenAI Python library
# version 0.28.1 or lower. import os


# API settings for Azure OpenAI
openai.api_type = "azure"
openai.api_base = "https://computationalsocialsciences.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = 'ADD_KEY'


system_prompt = "You will be asked to label a tweet according to some characteristics. Always answer with the name of the characteristic, follower by the label. E.g. Lexical Diversity: 5. Do not use any other language."
user_prompt = ('Lexical Diversity: Rate the variety and complexity of the vocabulary used in the tweet on a 7-point Likert scale (1 = very low diversity, 7 = very high diversity).\n'
            'Sentiment Analysis: Categorize the emotional tone of the tweet as either negative, neutral, or positive.\n'
            'Spelling and Grammar Quality: Assess the quality of spelling and grammar in the tweet on a 7-point Likert scale (1 = very poor quality, full of errors; 7 = excellent quality, no errors).\n'
            'Content Originality: Rate the originality of the content on a 7-point Likert scale (1 = completely unoriginal, likely plagiarized; 7 = highly original, unique content).\n'
            'Use of URLs: For URLs included in the tweet, classify the Tweet in one of the following categories: [`URL present + leads to reliable sources`, `URL present + leads to unreliable sources`,`URL present + reliability of sources not attainable`, `URL not present`]\n'
            'Hashtag Usage: Report on the use of hashtags in the tweet as one of the following categories: [`Yes - genuine usage`, `Yes - artificial usage`, `No`]\n'
            'Political Leanings: Provide an assessment on whether the Tweet`s content can be categorised as Liberal, Conservative, Independent or Cannot say.\n'
            'Bot Assessment: Provide an assessment on whether the tweet was likely written by a bot. Answer with Yes or No.\n'
            'Content Accuracy Assessment: Provide an assessment on a 7-point Likert representing the accuracy of the claims in the tweet, (1 = all claims in the tweet are verifiably inaccurate; 7 = all claims are true and verifiable).\n'
            'Conspiracy Assessment: Report on whether the tweet contributes to spread conspiracy theories [`Yes`,`No`,`Cannot say`] .\n'
            'Offensive Assessment: This tweet is offensive [`Yes`,`No`,`Cannot say`] .\n\n\n'
            'Tweet:')

# Function to parse the API response and return a dictionary
def parse_api_response(response):
    # Split the response by new lines
    lines = response.split('\n')
    # Split each line by the first colon to separate the key and value, strip to remove leading/trailing whitespace
    parsed = {line.split(':', 1)[0].strip(): line.split(':', 1)[1].strip() for line in lines if ':' in line}
    return parsed

def get_true_or_false(input_tweet):
    
    #prompt = f"Respond in the json format: {{'response': sentiment_classification}}\nText: {input_tweet}\nSentiment (true, false):"
    
    completion = openai.ChatCompletion.create(
        engine="GPT3",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=800,
        top_p =0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)
    
    response_text =  completion.choices[0].message['content'].strip()
    match = re.search(r'\btrue\b|\bfalse\b', response_text, flags=re.IGNORECASE)
    if match:
        truthfulness = match.group(0).lower()
    else:
        truthfulness = "Not Found"
    return {"tweet": input_tweet, "truthfulness": truthfulness}
    
